In [11]:
from Utility import df_ops, music_ops, vmd_ops, io_ops, cloud_ops, interpolate
import pandas as pd
import numpy as np
import pymeshio.common

pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.expand_frame_repr', False)  # Don't wrap to multiple pages
pd.set_option('max_colwidth', None)  # No max column width


#df_ops.saveDfToFeather(vmd_ops.getDfFromVmdFileName("play\\play.vmd"), "play\\play.feather")

In [12]:
# df = df_ops.loadDfFromFeather('play\\play.feather')
# df = vmd_ops.filterDataframeForValidBoneNames(df)
# vmd_ops.saveDfToVmdFile(df, 'play\\validBone.vmd')

In [13]:

def normalizePositionsToCenter(df):
  df_position_bones = df[df['name'].isin(['センター', '左足ＩＫ', '右足ＩＫ'])]
  lastFrame = df_ops.getLastFrame(df_position_bones)
  centerBone = 'センター'
  footIKBones = ['左足ＩＫ', '右足ＩＫ']
  boneList = [centerBone] + footIKBones

  lastBoneList = {boneName: {"frameNumber": None, "position": None, "rotation": None} for boneName in boneList}
  for boneName in lastBoneList.keys(): #init lastbonelist
    lastBoneList[boneName] = interpolate.getNextBoneOccurence(df_position_bones, boneName, 0)
  nextBoneList = {boneName: {"frameNumber": -1, "position": lastBoneList[boneName]['position'], "rotation": lastBoneList[boneName]['rotation']} for boneName in boneList}

  newFrames = []
  for currentFrameNumber in range(0, lastFrame + 1):
    frameDf = df_position_bones[df_position_bones['frame'] == currentFrameNumber]
    presentBonesOfBoneList = [boneName for boneName in boneList if boneName in frameDf['name'].values]
    print(currentFrameNumber, presentBonesOfBoneList)
    for boneName in presentBonesOfBoneList:
      if currentFrameNumber > nextBoneList[boneName]['frameNumber']:
        lastBoneList[boneName] = nextBoneList[boneName]
        nextBoneOccurenceValues = interpolate.getNextBoneOccurence(df_position_bones, boneName, currentFrameNumber)
        if nextBoneOccurenceValues is not None:
          nextBoneList[boneName] = nextBoneOccurenceValues
        else:
          nextBoneList[boneName] = {"frameNumber": lastFrame, "position": lastBoneList[boneName]['position'], "rotation": lastBoneList[boneName]['rotation']}



df = df_ops.loadDfFromFeather('play\\play.feather')
df = vmd_ops.filterDataframeForCoreBoneNames(df)
df = normalizePositionsToCenter(df)

0 ['センター', '左足ＩＫ', '右足ＩＫ']
1 ['センター', '左足ＩＫ', '右足ＩＫ']
2 ['センター', '左足ＩＫ', '右足ＩＫ']
3 ['センター', '左足ＩＫ', '右足ＩＫ']
4 ['センター', '左足ＩＫ', '右足ＩＫ']
5 ['センター', '左足ＩＫ', '右足ＩＫ']
6 ['センター', '左足ＩＫ', '右足ＩＫ']
7 ['センター', '左足ＩＫ', '右足ＩＫ']
8 ['センター', '左足ＩＫ', '右足ＩＫ']
9 ['センター', '左足ＩＫ', '右足ＩＫ']
10 ['センター', '左足ＩＫ', '右足ＩＫ']
11 ['センター', '左足ＩＫ', '右足ＩＫ']
12 ['センター', '左足ＩＫ', '右足ＩＫ']
13 ['センター', '左足ＩＫ', '右足ＩＫ']
14 ['センター', '左足ＩＫ', '右足ＩＫ']
15 ['センター', '左足ＩＫ', '右足ＩＫ']
16 ['センター', '左足ＩＫ', '右足ＩＫ']
17 ['センター', '左足ＩＫ', '右足ＩＫ']
18 ['センター', '左足ＩＫ', '右足ＩＫ']
19 ['センター', '左足ＩＫ', '右足ＩＫ']
20 ['センター', '左足ＩＫ', '右足ＩＫ']
21 ['センター', '左足ＩＫ', '右足ＩＫ']
22 ['センター', '左足ＩＫ', '右足ＩＫ']
23 ['センター', '左足ＩＫ', '右足ＩＫ']
24 ['センター', '左足ＩＫ', '右足ＩＫ']
25 ['センター', '左足ＩＫ', '右足ＩＫ']
26 ['センター', '左足ＩＫ', '右足ＩＫ']
27 ['センター', '左足ＩＫ', '右足ＩＫ']
28 ['センター', '左足ＩＫ', '右足ＩＫ']
29 ['センター', '左足ＩＫ', '右足ＩＫ']
30 ['センター', '左足ＩＫ', '右足ＩＫ']
31 ['センター', '左足ＩＫ', '右足ＩＫ']
32 ['センター', '左足ＩＫ', '右足ＩＫ']
33 ['センター', '左足ＩＫ', '右足ＩＫ']
34 ['センター', '左足ＩＫ', '右足ＩＫ']
35 ['センター', '左足ＩＫ', '右足ＩＫ']
36

In [14]:
# def printInfo(wav, vmd, foldername):
#   print(foldername)
#   vmd_ops.printVmdDetails(vmd=vmd)

# io_ops.apply_func_to_inputData(printInfo, 'inputData')